<a href="https://colab.research.google.com/github/Thejaswi05/research_agent_demo/blob/main/ODSC_AI_Research_Assistant_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem**


---


For every scientist, academic, grad student, and engineer - literature review is a daunting task. We only wish we could automate getting insights and simplify work of finding papers and summarizing them. With advent of AI that is a distinct possibility. In this demo we attempt to build a Research Assistant  agent for this.




#### **DISCLAIMER:**
THIS IS NOT A NOVEL IDEA / PROBLEM, WE ARE USING THIS AS AN EXAMPLE TO EXPLAIN CONCEPTS OF BUILDING AGENTS. AGENT BUILT IS NOT MEANT FOR PRODUCTION USE. USE AT YOUR OWN RISK.


---



# **Building The Research Agent**


---


#### **LAYOUT:**
In this notebook we will be working towards building an agent in a step by step manner. Anthropic released a blog post about build AI Agents and discussed some very important concepts - [Building Effective Agents by Anthropic]( https://www.anthropic.com/engineering/building-effective-agents). We will walk through some critical patterns that help us to build the agent.


**Optional:** At the end of the notebook, there is an optional section that shows a succinct version of the agent using the LangGraph constructs to simplify the more education implementation. This would be useful for folks who want to see a succinct implementation.

Following are the patterns we will explore:
1. The Agumented Model
2. Prompt Chaining
3. Routing
4. Parallelization - **Not Implemented**
5. Orchestrator-workers - **Not Implemented**
6. Evaluator-optimizer - **Not Implemented**
7. The Agent

**Image Credits:** We are borrowing some of the images from the [Building Effective Agents by Anthropic]( https://www.anthropic.com/engineering/building-effective-agents) blog. Copyright for those images is with original publishers, we are reusing with gratitute.



---







# **Infrastructure Code**

## Install

In [1]:
!pip install langchain langchain-core langchain-groq langgraph arxiv PyPDF2 requests langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Imports

In [2]:
# Import necessary libraries
import os
from typing import List, Dict, Any, Optional
import arxiv
from langchain.agents.agent import AgentExecutor, AgentAction, AgentFinish
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import Tool, StructuredTool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
#from langgraph.prebuilt import ToolExecutor
import requests
import PyPDF2
from io import BytesIO
import re

In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the Groq API key
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Check if the key was found
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in .env file")

print("Groq API key loaded successfully!")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("OpenAI API key loaded successfully!")




Groq API key loaded successfully!
OpenAI API key loaded successfully!


## Tools

Let us create a set of tools which we will give to our LLM to use during its ReAct loop for the agent.


We will keep using these tools again and again, to build the patterns called out above. This illustrates the reusability of the tools in AI Application building processes.

In a enterprise / production environment - tools will be your existing internal APIs that are datasources that you would like to include as tools in our AI app.

---



In [5]:
# 1. Simple ArXiv Search Tool
def search_arxiv(query, max_results=5):
    """Search for papers on arXiv and return basic info."""
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results)

    results = []
    for paper in client.results(search):
        results.append({
            "id": paper.entry_id.split("/")[-1],
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "url": paper.pdf_url
        })
    return results

In [6]:
# 2. Simple Paper Downloader
def download_paper(paper_id):
    """Download a paper from arXiv by ID and extract text."""
    search = arxiv.Search(id_list=[paper_id])
    client = arxiv.Client()
    paper = next(client.results(search))

    response = requests.get(paper.pdf_url)
    pdf_file = BytesIO(response.content)
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    return text[:5000]  # Return first 5000 chars for simplicity

In [7]:
# 3. Simple Summarizer
def summarize_text(text, groq_api_key):
    """Summarize text using Groq."""
    model = ChatGroq(api_key=groq_api_key, model_name="meta-llama/llama-4-maverick-17b-128e-instruct")
    prompt = f"Summarize this academic paper: {text[:15000]}"
    return model.invoke(prompt).content

In [8]:
def create_tools(groq_api_key: str):
    """Create the tools for our agent."""

    tools = [
        Tool(
            name="search_arxiv",
            description="Search for academic papers on arXiv. Input is a search query.",
            func=lambda q: search_arxiv(q)
        ),
        Tool(
            name="download_paper",
            description="Download a paper from arXiv and extract its text. Input is a paper ID.",
            func=lambda id: download_paper(id)
        ),
        Tool(
            name="summarize_text",
            description="Summarize a piece of text. Input is the text to summarize.",
            func=lambda t: summarize_text(t, groq_api_key)
        )
    ]

    return tools

## The Augmented Model

![Augmented LLM](./demo_pics/augmented_llm.png)
This is the fundamental building block for building an Agents. The LLM is supplimented with tools and memory. Since we are using LnagChain as our framework, memory construct is inbuilt for us. We are treating search as equivalent to retreival in this case.

Let us build a simple version of this pattern.


---



In [88]:
# First, create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that can use tools to answer user questions about research papers."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

def setup_augmented_llm(groq_api_key):
    # Define the tools
    tools = create_tools(groq_api_key)

    # Create the LLM
    llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")

    # Create the agent with tools
    agent = create_openai_tools_agent(llm, tools, prompt)

    # Create an agent executor
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor

# 5. Run the system
def run_augmented_llm(query, groq_api_key):
    agent_executor = setup_augmented_llm(groq_api_key)
    result = agent_executor.invoke({"input": query})
    return result["output"]

#### The Augmented LLM - Demo

In [89]:
query = "What are the latest developments in quantum computing?"
response = run_augmented_llm(query, groq_api_key)
print(response)



> Entering new AgentExecutor chain...

Invoking: `search_arxiv` with `latest developments quantum computing`


[{'id': '0305472v2', 'title': 'Arbitrage risk induced by transaction costs', 'authors': ['E. W. Piotrowski', 'J. Sladkowski'], 'url': 'http://arxiv.org/pdf/cond-mat/0305472v2'}, {'id': '2302.00001v1', 'title': 'A Living Review of Quantum Computing for Plasma Physics', 'authors': ['Óscar Amaro', 'Diogo Cruz'], 'url': 'http://arxiv.org/pdf/2302.00001v1'}, {'id': '2502.08925v1', 'title': 'Quantum Software Engineering and Potential of Quantum Computing in Software Engineering Research: A Review', 'authors': ['Ashis Kumar Mandal', 'Md Nadim', 'Chanchal K. Roy', 'Banani Roy', 'Kevin A. Schneider'], 'url': 'http://arxiv.org/pdf/2502.08925v1'}, {'id': '2407.16296v1', 'title': 'Quantum Computing for Climate Resilience and Sustainability Challenges', 'authors': ['Kin Tung Michael Ho', 'Kuan-Cheng Chen', 'Lily Lee', 'Felix Burt', 'Shang Yu', 'Po-Heng', 'Lee'], 'url': 'http://arxiv.org/

## Prompt Chaining

![Prompt Chaining](./demo_pics/prompt_chain.png)

This workflow pattern chains output of one LLM Call / Tool to the other one.

The research assistant is the orchestrator that coordinates the search and synthesis. search_llm and synthesis_llm are the two LLMs in this workflow. The output of search_llm is fed into the input of synthesis_llm. search_llm is used to convert the user's question into a search query using openai. synthesis_llm is used to synthesize the results of the search using groq.



---



### Setup Prompts

In [85]:
def setup_prompt_chain(groq_api_key, openai_api_key):
    """
    Setup a chain of prompts where the output of one LLM call is fed into another.
    This demonstrates the Prompt Chaining pattern.
    """
    # Create the LLMs
    search_llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
    synthesis_llm = ChatGroq(api_key=groq_api_key, model_name="meta-llama/llama-4-maverick-17b-128e-instruct")

    # Create the tools
    tools = create_tools(groq_api_key)

    # STEP 1: Create the search agent prompt
    search_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a research assistant that helps find relevant papers on arXiv.
Your job is to convert a user's research question into the best possible search query for arXiv.
Focus on extracting key technical terms and concepts.
Respond ONLY with the search query, nothing else."""),
        ("human", "{input}")
    ])

    # STEP 2: Create the paper selection agent prompt
    selection_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a research assistant that helps select the most relevant paper from search results.
Given a list of papers and the original research question, select the SINGLE most relevant paper ID.
Respond ONLY with the paper ID, nothing else."""),
        ("human", "Research question: {original_query}\nSearch results: {search_results}")
    ])

    # STEP 3: Create the analysis agent prompt
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a research expert that analyzes academic papers.
Given the text of a paper, identify the key methodologies, findings, and implications.
Focus on how this paper addresses the user's original question."""),
        ("human", "Original question: {original_query}\nPaper text: {paper_text}")
    ])

    # STEP 4: Create the final synthesis prompt
    synthesis_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful AI research assistant that synthesizes information into clear,
concise responses. Create a comprehensive yet accessible answer to the user's question based on the
technical analysis provided."""),
        ("human", "Original question: {original_query}\nTechnical analysis: {analysis}")
    ])

    return {
        "search_llm": search_llm,
        "synthesis_llm": synthesis_llm,
        "tools": tools,
        "prompts": {
            "search": search_prompt,
            "selection": selection_prompt,
            "analysis": analysis_prompt,
            "synthesis": synthesis_prompt
        }
    }

### Setup Chaining

In [86]:
def run_prompt_chain(query, groq_api_key, openai_api_key):
    """
    Execute the entire prompt chain to answer a research question.
    This demonstrates how outputs from one LLM call are fed as inputs to the next.
    """
    # Setup the chain components
    chain = setup_prompt_chain(groq_api_key, openai_api_key)
    search_llm = chain["search_llm"]
    synthesis_llm = chain["synthesis_llm"]
    tools = chain["tools"]
    prompts = chain["prompts"]

    print("📝 STARTING PROMPT CHAIN")
    print(f"📋 Original query: {query}")

    # STEP 1: Convert question to search query
    print("\n🔍 Step 1: Converting question to optimal search query...")
    search_query = search_llm.invoke(prompts["search"].format(input=query)).content
    print(f"🔍 Generated search query: {search_query}")

    # STEP 2: Search for papers
    print("\n📚 Step 2: Searching for papers on arXiv...")
    search_results = search_arxiv(search_query)
    print(f"📚 Found {len(search_results)} papers")

    # STEP 3: Select the most relevant paper
    print("\n🔎 Step 3: Selecting the most relevant paper...")
    paper_id = search_llm.invoke(
        prompts["selection"].format(
            original_query=query,
            search_results=search_results
        )
    ).content
    print(f"🔎 Selected paper ID: {paper_id}")

    # STEP 4: Download the selected paper
    print("\n📄 Step 4: Downloading the selected paper...")
    paper_text = download_paper(paper_id)
    print(f"📄 Downloaded {len(paper_text)} characters of paper text")

    # STEP 5: Analyze the paper
    print("\n🧪 Step 5: Analyzing the paper...")
    analysis = search_llm.invoke(
        prompts["analysis"].format(
            original_query=query,
            paper_text=paper_text
        )
    ).content
    print(f"🧪 Analysis complete")

    # STEP 6: Final synthesis
    print("\n🔮 Step 6: Synthesizing final response...")
    final_response = synthesis_llm.invoke(
        prompts["synthesis"].format(
            original_query=query,
            analysis=analysis
        )
    ).content

    print("\n✅ PROMPT CHAIN COMPLETE")

    return {
        "search_query": search_query,
        "selected_paper": next((p for p in search_results if p["id"] == paper_id), None),
        "analysis": analysis,
        "response": final_response
    }

In [87]:
# Example usage
def run_example():
    # Load API keys from environment
    # groq_api_key = os.getenv("GROQ_API_KEY")
    # openai_api_key = os.getenv("OPENAI_API_KEY")

    if not groq_api_key or not openai_api_key:
        raise ValueError("API keys not found. Make sure GROQ_API_KEY and OPENAI_API_KEY are set.")

    # Run the prompt chain
    query = "What are the latest breakthroughs in quantum error correction?"
    result = run_prompt_chain(query, groq_api_key, openai_api_key)

    # Output the final response
    print("\n----- FINAL RESPONSE -----")
    print(result["response"])
    print("\n--------------------------")

    return result

run_example()

📝 STARTING PROMPT CHAIN
📋 Original query: What are the latest breakthroughs in quantum error correction?

🔍 Step 1: Converting question to optimal search query...
🔍 Generated search query: quantum error correction latest breakthroughs recent developments

📚 Step 2: Searching for papers on arXiv...
📚 Found 5 papers

🔎 Step 3: Selecting the most relevant paper...
🔎 Selected paper ID: 2412.20380v1

📄 Step 4: Downloading the selected paper...
📄 Downloaded 5000 characters of paper text

🧪 Step 5: Analyzing the paper...
🧪 Analysis complete

🔮 Step 6: Synthesizing final response...

✅ PROMPT CHAIN COMPLETE

----- FINAL RESPONSE -----
The latest breakthroughs in quantum error correction (QEC) are driven by the integration of artificial intelligence (AI) and machine learning (ML) techniques with traditional QEC methods. Recent studies, as highlighted in a comprehensive survey of over 150 papers, have shown that AI-driven QEC approaches outperform conventional methods in terms of efficiency and 

{'search_query': 'quantum error correction latest breakthroughs recent developments',
 'selected_paper': {'id': '2412.20380v1',
  'title': 'Artificial Intelligence for Quantum Error Correction: A Comprehensive Review',
  'authors': ['Zihao Wang', 'Hao Tang'],
  'url': 'http://arxiv.org/pdf/2412.20380v1'},
 'analysis': "Below is an analysis that highlights the paper’s methodologies, key findings, and implications with a focus on addressing breakthroughs in quantum error correction (QEC):\n\n1. Methodologies:\n\u2003• The paper is a comprehensive survey that reviews over 150 studies, with a particular focus on the integration of artificial intelligence (AI) and machine learning (ML) techniques into QEC.\n\u2003• It categorizes the AI/ML methods into several groups—unsupervised, supervised, semi-supervised, and reinforcement learning—exploring how these techniques enhance the detection and correction of quantum errors.\n\u2003• A comparative approach is used: traditional QEC methods such 

## Routing

![Routing](./demo_pics/llm_router.png)

Routing is a pattern where the agent analyzes the query and directs it to the appropriate specialized pathway.

---

In [90]:
class RouterAgent:
    """
    A router that analyzes the query and directs it to the appropriate specialized pathway.
    """
    def __init__(self, openai_api_key):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.router_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a query classifier that determines the type of research question.
Analyze the user's research question and classify it into EXACTLY ONE of the following categories:
1. "latest_developments" - Questions about recent advancements or state-of-the-art in a field
2. "deep_dive" - Questions requiring in-depth analysis of specific techniques or methodologies
3. "comparison" - Questions comparing different approaches, methods, or theories
4. "application" - Questions about applying research to specific domains or problems

Respond ONLY with the category name in lowercase, nothing else."""),
            ("human", "{query}")
        ])

    def route(self, query):
        """Determine which specialized pathway should handle this query."""
        response = self.llm.invoke(self.router_prompt.format(query=query))
        query_type = response.content.strip().lower()

        # Validate that the response is one of our expected categories
        valid_types = ["latest_developments", "deep_dive", "comparison", "application"]
        if query_type not in valid_types:
            # Default to latest_developments if classification fails
            query_type = "latest_developments"

        return query_type

class LatestDevelopmentsAgent:
    """Specialized agent for handling queries about recent developments."""
    def __init__(self, openai_api_key, groq_api_key, tools):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.tools = tools
        self.groq_api_key = groq_api_key

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant specializing in the latest developments and
state-of-the-art research. Your goal is to find the most recent papers on a topic and
summarize the cutting-edge advancements."""),
            ("human", "{query}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def process(self, query):
        agent = create_openai_tools_agent(self.llm, self.tools, self.prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        result = agent_executor.invoke({"query": query})
        return result["output"]

class DeepDiveAgent:
    """Specialized agent for handling deep-dive queries."""
    def __init__(self, openai_api_key, groq_api_key, tools):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.tools = tools
        self.groq_api_key = groq_api_key

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant specializing in deep technical analysis.
Your goal is to find and thoroughly analyze papers on specific techniques or methodologies.
Focus on understanding the core principles, implementation details, and theoretical foundations."""),
            ("human", "{query}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def process(self, query):
        agent = create_openai_tools_agent(self.llm, self.tools, self.prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        result = agent_executor.invoke({"query": query})
        return result["output"]

class ComparisonAgent:
    """Specialized agent for handling comparison queries."""
    def __init__(self, openai_api_key, groq_api_key, tools):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.tools = tools
        self.groq_api_key = groq_api_key

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant specializing in comparative analysis.
Your goal is to find papers that compare different approaches, methods, or theories and
analyze the strengths and weaknesses of each. Create a balanced assessment that highlights
the key differences."""),
            ("human", "{query}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def process(self, query):
        agent = create_openai_tools_agent(self.llm, self.tools, self.prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        result = agent_executor.invoke({"query": query})
        return result["output"]

class ApplicationAgent:
    """Specialized agent for handling application queries."""
    def __init__(self, openai_api_key, groq_api_key, tools):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.tools = tools
        self.groq_api_key = groq_api_key

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant specializing in practical applications.
Your goal is to find papers that demonstrate how research is applied to real-world problems
and domains. Focus on implementation details, case studies, and practical outcomes."""),
            ("human", "{query}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def process(self, query):
        agent = create_openai_tools_agent(self.llm, self.tools, self.prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        result = agent_executor.invoke({"query": query})
        return result["output"]

def setup_routing_system(openai_api_key, groq_api_key):
    """Set up the complete routing system with specialized agents."""
    tools = create_tools(groq_api_key)

    # Create the router and specialized agents
    router = RouterAgent(openai_api_key)
    agents = {
        "latest_developments": LatestDevelopmentsAgent(openai_api_key, groq_api_key, tools),
        "deep_dive": DeepDiveAgent(openai_api_key, groq_api_key, tools),
        "comparison": ComparisonAgent(openai_api_key, groq_api_key, tools),
        "application": ApplicationAgent(openai_api_key, groq_api_key, tools)
    }

    return router, agents


In [91]:
def run_routing_system(query, openai_api_key, groq_api_key):
    """Run the routing system end-to-end."""
    router, agents = setup_routing_system(openai_api_key, groq_api_key)

    print("🚦 ROUTING SYSTEM INITIATED")
    print(f"📋 Query: {query}")

    # Step 1: Route the query
    print("\n🔀 Routing query...")
    query_type = router.route(query)
    print(f"🔀 Query classified as: {query_type}")

    # Step 2: Process with the appropriate agent
    print(f"\n🔬 Processing with specialized {query_type} agent...")
    response = agents[query_type].process(query)

    print("\n✅ ROUTING COMPLETE")

    return {
        "query_type": query_type,
        "response": response
    }

# Example usage
def run_example():
    # Load API keys from environment
    # groq_api_key = os.getenv("GROQ_API_KEY")
    # openai_api_key = os.getenv("OPENAI_API_KEY")

    if not groq_api_key or not openai_api_key:
        raise ValueError("API keys not found. Make sure GROQ_API_KEY and OPENAI_API_KEY are set.")

    # Example queries for different types
    example_queries = {
        "latest_developments": "What are the latest advancements in quantum computing?",
        "deep_dive": "How do transformers handle attention mechanisms in deep learning?",
        "comparison": "Compare LSTM and transformer approaches for sequence modeling",
        "application": "How is reinforcement learning applied to robotics?"
    }

    # Choose one query to run
    query = example_queries["deep_dive"]

    # Run the routing system
    result = run_routing_system(query, openai_api_key, groq_api_key)

    # Output the final response
    print("\n----- FINAL RESPONSE -----")
    print(result["response"])
    print("\n--------------------------")

    return result

run_example()

🚦 ROUTING SYSTEM INITIATED
📋 Query: How do transformers handle attention mechanisms in deep learning?

🔀 Routing query...
🔀 Query classified as: deep_dive

🔬 Processing with specialized deep_dive agent...


> Entering new AgentExecutor chain...
Transformers revolutionized deep learning by introducing the self-attention mechanism, which allows the model to weigh the importance of different tokens in the sequence when generating representations. Here’s a detailed look at how they handle attention:

1. Self-Attention with Query, Key, and Value Vectors:
   • Each input token is transformed into three vectors: a query (Q), a key (K), and a value (V) through learned linear projections.
   • The attention mechanism computes a score between pairs of tokens by taking the dot product of a token's query with the key of another token. These scores indicate the relative importance or alignment between tokens.

2. Scaled Dot-Product Attention:
   • The raw dot-product scores are scaled by dividing b

{'query_type': 'deep_dive',
 'response': "Transformers revolutionized deep learning by introducing the self-attention mechanism, which allows the model to weigh the importance of different tokens in the sequence when generating representations. Here’s a detailed look at how they handle attention:\n\n1. Self-Attention with Query, Key, and Value Vectors:\n   • Each input token is transformed into three vectors: a query (Q), a key (K), and a value (V) through learned linear projections.\n   • The attention mechanism computes a score between pairs of tokens by taking the dot product of a token's query with the key of another token. These scores indicate the relative importance or alignment between tokens.\n\n2. Scaled Dot-Product Attention:\n   • The raw dot-product scores are scaled by dividing by the square root of the dimension of the key vectors (usually denoted as √(dₖ)). This scaling helps mitigate issues with very large values in high-dimensional spaces.\n   • A softmax function is 

## The Agent

![Anthropic Prompt Chaining](https://drive.google.com/uc?export=view&id=1_ERsFpGuxGUq_lkfuoW0O5dc9nrSXoSy)

Most flexible but difficult to get it right pattern, where you are relying on LLM (usually more than one) to provide a reasoned plan of action, observe and evaluate the results of the actions from environment, reason and decide on next steps including stopping the computation.

---

LLM as Orchstrator & Evaluator

In [14]:
from enum import Enum
import json
from typing import List, Dict, Any, Optional, Union
class AgentAction(Enum):
    """Enum for different possible agent actions."""
    USE_TOOL = "use_tool"
    ROUTE = "route"
    FINAL_ANSWER = "final_answer"

class ResearchPath(Enum):
    """Enum for different research paths."""
    OVERVIEW = "overview"
    DEEP_DIVE = "deep_dive"
    COMPARISON = "comparison"
    TECHNICAL = "technical"

# Define the schema for LLM response using Pydantic
class ToolActionInput(BaseModel):
    """Schema for tool action input."""
    tool_name: str = Field(description="Name of the tool to use")
    tool_input: str = Field(description="Input to provide to the tool")

class RouteActionInput(BaseModel):
    """Schema for routing action input."""
    path: str = Field(description="Research path to take")

class FinalAnswerInput(BaseModel):
    """Schema for final answer input."""
    answer: str = Field(description="Final answer to the query")

class AgentDecision(BaseModel):
    """Schema for agent decision."""
    thought: str = Field(description="Detailed reasoning about the current state and what to do next")
    action: str = Field(description="Action to take: use_tool, route, or final_answer")
    action_input: Union[ToolActionInput, RouteActionInput, FinalAnswerInput] = Field(
        description="Input for the action"
    )

class AgentState:
    """State container for the agent's reasoning process."""
    def __init__(self, query: str):
        self.query = query
        self.current_path: Optional[ResearchPath] = None
        self.tool_results: List[Dict[str, Any]] = []
        self.working_memory: List[str] = []
        self.final_answer: Optional[str] = None
        self.conversation_history: List[Dict[str, Any]] = []

    def add_tool_result(self, tool_name: str, tool_input: str, tool_output: Any) -> None:
        """Add the result of a tool call to the state."""
        self.tool_results.append({
            "tool_name": tool_name,
            "tool_input": tool_input,
            "tool_output": tool_output
        })

    def add_to_memory(self, thought: str) -> None:
        """Add a thought to working memory."""
        self.working_memory.append(thought)

    def set_path(self, path: ResearchPath) -> None:
        """Set the current research path."""
        self.current_path = path

    def set_final_answer(self, answer: str) -> None:
        """Set the final answer."""
        self.final_answer = answer

    def add_message(self, role: str, content: str) -> None:
        """Add a message to the conversation history."""
        self.conversation_history.append({
            "role": role,
            "content": content
        })

    def to_dict(self) -> Dict[str, Any]:
        """Convert the state to a dictionary for serialization."""
        return {
            "query": self.query,
            "current_path": self.current_path.value if self.current_path else None,
            "tool_results": self.tool_results,
            "working_memory": self.working_memory,
            "final_answer": self.final_answer,
        }

class ResearchAgent:
    """An agent that can dynamically make decisions about research queries."""

    def __init__(self, openai_api_key: str, groq_api_key: str):
        self.llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")
        self.groq_api_key = groq_api_key
        self.tools = create_tools(groq_api_key)

        # Create the JSON output parser
        self.parser = JsonOutputParser(pydantic_object=AgentDecision)

        # Create the decision prompt template with LangChain's JSON parser format instructions
        self.decision_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant agent capable of dynamically making decisions about how to answer research questions.
You must explicitly reason through THREE types of decisions:

1. ROUTING - Determine which research path would best address the query:
   - OVERVIEW: For broad, introductory questions needing a general summary
   - DEEP_DIVE: For questions requiring in-depth focus on a specific topic
   - COMPARISON: For questions comparing multiple approaches or methods
   - TECHNICAL: For questions requiring detailed technical explanations

2. TOOL SELECTION - Determine which tool to use next from:
   - search_arxiv: Search for academic papers (input: search query)
   - download_paper: Get the text of a specific paper (input: paper ID)
   - summarize_text: Summarize a piece of text (input: text to summarize)

3. FINAL ANSWER - Determine if you have enough information to provide a final answer

For each decision, you MUST use the following process:
1. Think through the options step-by-step
2. Consider the query and current state
3. Make an explicit choice
4. Explain your reasoning

{format_instructions}"""),
            ("human", """
QUERY: {query}

CURRENT PATH: {current_path}

TOOL RESULTS:
{tool_results}

WORKING MEMORY:
{working_memory}

Based on the above, what should I do next? Think through whether I should choose a research path, use a tool, or provide a final answer.
""")
        ])

    def _format_tool_results(self, tool_results: List[Dict[str, Any]]) -> str:
        """Format tool results for the prompt."""
        if not tool_results:
            return "No tools have been used yet."

        result_strings = []
        for i, result in enumerate(tool_results):
            result_str = f"Tool {i+1}: {result['tool_name']}\n"
            result_str += f"Input: {result['tool_input']}\n"

            # Format the output depending on the tool
            if result['tool_name'] == "search_arxiv":
                papers = result['tool_output']
                result_str += "Output (Papers):\n"
                for j, paper in enumerate(papers):
                    result_str += f"  Paper {j+1}: {paper['title']}\n"
                    result_str += f"  ID: {paper['id']}\n"
                    result_str += f"  Authors: {', '.join(paper['authors'])}\n"
                    result_str += f"  URL: {paper['url']}\n\n"
            elif result['tool_name'] == "download_paper":
                # Truncate long paper texts
                paper_text = result['tool_output']
                if len(paper_text) > 500:
                    paper_text = paper_text[:500] + "... [text truncated]"
                result_str += f"Output (Paper Text):\n{paper_text}\n\n"
            elif result['tool_name'] == "summarize_text":
                result_str += f"Output (Summary):\n{result['tool_output']}\n\n"

            result_strings.append(result_str)

        return "\n".join(result_strings)

    def _format_working_memory(self, working_memory: List[str]) -> str:
        """Format working memory for the prompt."""
        if not working_memory:
            return "No thoughts recorded yet."

        memory_strings = []
        for i, thought in enumerate(working_memory):
            memory_strings.append(f"Thought {i+1}: {thought}")

        return "\n".join(memory_strings)

    def _get_next_action(self, state: AgentState) -> Dict[str, Any]:
      """Get the next action from the LLM."""
      # Format the state for the prompt
      prompt_args = {
          "query": state.query,
          "current_path": state.current_path.value if state.current_path else "Not selected yet",
          "tool_results": self._format_tool_results(state.tool_results),
          "working_memory": self._format_working_memory(state.working_memory),
          "format_instructions": self.parser.get_format_instructions()
      }
      
      # Get the LLM's decision
      response = self.llm.invoke(self.decision_prompt.format(**prompt_args))
      
      # Parse the JSON response
      try:
          # First, parse the raw text into a Python dictionary
          response_text = response.content
          
          # Try to find JSON in the response
          import re
          import json
          
          # Look for JSON patterns
          match = re.search(r'(\{.*\})', response_text, re.DOTALL)
          if match:
              json_str = match.group(1)
              # Parse JSON string into Python dictionary
              action_data = json.loads(json_str)
              
              # Now we have a dictionary with the parsed data
              return {
                  "thought": action_data.get("thought", "No reasoning provided"),
                  "action": action_data.get("action", "final_answer"),
                  "action_input": action_data.get("action_input", {"answer": "I don't have enough information to answer."})
              }
          else:
              # If no JSON found
              return {
                  "thought": "Failed to extract JSON from response",
                  "action": "final_answer",
                  "action_input": {
                      "answer": "I encountered an error in my reasoning process."
                  }
              }
              
      except Exception as e:
          # Better error handling
          print(f"Error parsing LLM response: {e}")
          print(f"Problem response: {response_text}")
          
          # Fall back to a predetermined structure
          return {
              "thought": f"There was an error parsing my previous response: {str(e)}",
              "action": "final_answer",
              "action_input": {
                  "answer": "I encountered a technical error in my reasoning process. Let me provide a direct answer based on my knowledge: Transformers and LSTMs differ primarily in how they process sequential data - LSTMs process data sequentially while transformers can process the entire sequence at once through self-attention mechanisms."
              }
          }

    def _execute_tool(self, tool_name: str, tool_input: str) -> Any:
        """Execute a tool and return the result."""
        # Find the tool
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            return f"Error: Tool '{tool_name}' not found."

        # Execute the tool
        try:
            result = tool.func(tool_input)
            return result
        except Exception as e:
            return f"Error executing tool: {str(e)}"

    def _route_query(self, path_name: str) -> ResearchPath:
        """Route the query to a research path."""
        try:
            return ResearchPath(path_name)
        except ValueError:
            # Default to overview if invalid path
            return ResearchPath.OVERVIEW

    def process_query(self, query: str, max_steps: int = 10) -> Dict[str, Any]:
        """
        Process a research query using the agent's reasoning loop.

        Args:
            query: The research query to process
            max_steps: Maximum number of reasoning steps to perform

        Returns:
            A dictionary containing the final answer and the agent's state
        """
        # Initialize the agent state
        state = AgentState(query)

        print("🤖 RESEARCH AGENT INITIATED")
        print(f"📋 Query: {query}")

        # Main reasoning loop
        for step in range(max_steps):
            print(f"\n🔄 Step {step+1}: Reasoning...")

            # Get the next action
            action_data = self._get_next_action(state)

            # Add the thought to working memory
            thought = action_data.get("thought", "No reasoning provided")
            state.add_to_memory(thought)
            print(f"💭 Thought: {thought}")

            # Parse the action
            action = action_data.get("action")
            action_input = action_data.get("action_input", {})

            if action == AgentAction.USE_TOOL.value:
                # Execute the tool
                tool_name = action_input.get("tool_name")
                tool_input = action_input.get("tool_input")

                print(f"🔧 Using Tool: {tool_name}")
                print(f"📥 Tool Input: {tool_input}")

                tool_output = self._execute_tool(tool_name, tool_input)
                state.add_tool_result(tool_name, tool_input, tool_output)

                print(f"📤 Tool Output: {type(tool_output)} with {len(str(tool_output))} chars")

            elif action == AgentAction.ROUTE.value:
                # Set the research path
                path_name = action_input.get("path")
                path = self._route_query(path_name)
                state.set_path(path)

                print(f"🔀 Routing to Path: {path.value}")

            elif action == AgentAction.FINAL_ANSWER.value:
                # Set the final answer
                answer = action_input.get("answer")
                state.set_final_answer(answer)

                print(f"✅ Final Answer Ready")
                break

            else:
                # Handle unknown action
                print(f"⚠️ Unknown Action: {action}")
                state.set_final_answer(f"I encountered an error in my reasoning process. Unknown action: {action}")
                break
        else:
            # Handle max steps reached
            state.set_final_answer("I've reached the maximum number of reasoning steps without finding a complete answer. Here's what I've learned so far: " +
                                 "\n\n".join(state.working_memory[-3:]))
            print("⚠️ Maximum steps reached without final answer")

        print("\n✨ REASONING COMPLETE")

        return {
            "final_answer": state.final_answer,
            "state": state.to_dict()
        }


In [15]:
# Example usage
def run_example():
    # Load API keys from environment
    # groq_api_key = os.getenv("GROQ_API_KEY")
    # openai_api_key = os.getenv("OPENAI_API_KEY")

    if not groq_api_key or not openai_api_key:
        raise ValueError("API keys not found. Make sure GROQ_API_KEY and OPENAI_API_KEY are set.")

    # Create the agent
    agent = ResearchAgent(openai_api_key, groq_api_key)

    # Example query
    query = "What are the differences between transformers and LSTMs for NLP tasks?"

    # Process the query
    result = agent.process_query(query)

    # Output the final answer
    print("\n----- FINAL ANSWER -----")
    print(result["final_answer"])
    print("\n--------------------------")

    return result

run_example()

🤖 RESEARCH AGENT INITIATED
📋 Query: What are the differences between transformers and LSTMs for NLP tasks?

🔄 Step 1: Reasoning...
Error parsing LLM response: 'dict' object has no attribute 'thought'
Problem response: {
  "thought": "The query asks for a comparison between transformers and LSTMs for NLP tasks. This is clearly a COMPARISON type question. Given that I have sufficient background knowledge about these architectures, I can provide a comprehensive answer directly without needing to consult additional resources or external tools.",
  "action": "final_answer",
  "action_input": {
    "answer": "Transformers and LSTMs differ significantly in their design and how they process sequential data for NLP tasks. Here are the key differences:\n\n1. Architectural Approach:\n   - LSTMs (Long Short-Term Memory networks) are a type of recurrent neural network (RNN) that process data sequentially. They maintain and update a hidden state that evolves with each time step, which makes them inh

{'final_answer': 'I encountered a technical error in my reasoning process. Let me provide a direct answer based on my knowledge: Transformers and LSTMs differ primarily in how they process sequential data - LSTMs process data sequentially while transformers can process the entire sequence at once through self-attention mechanisms.',
 'state': {'query': 'What are the differences between transformers and LSTMs for NLP tasks?',
  'current_path': None,
  'tool_results': [],
  'working_memory': ["There was an error parsing my previous response: 'dict' object has no attribute 'thought'"],
  'final_answer': 'I encountered a technical error in my reasoning process. Let me provide a direct answer based on my knowledge: Transformers and LSTMs differ primarily in how they process sequential data - LSTMs process data sequentially while transformers can process the entire sequence at once through self-attention mechanisms.'}}

## [Optional] A Succinct Version

This implementation of the agent uses LangGraph framework to make the above verbose version (which was more for illustrative perposes) more succinct while not loosing any functionality.

This goes on to show that frameworks like `langgraph` abstract common boiler plate code into the frameworks and provide important prebuilt constructs like OutputParser to simplify the agent building and working with LLMs.


---



In [31]:
# ---- Define tool schemas for better structured input ----

class SearchArxivInput(BaseModel):
    """Input schema for searching arXiv."""
    query: str = Field(description="The search query for arXiv papers")
    max_results: int = Field(default=5, description="Maximum number of results")

class DownloadPaperInput(BaseModel):
    """Input schema for downloading a paper."""
    paper_id: str = Field(description="The arXiv paper ID to download")

class SummarizeTextInput(BaseModel):
    """Input schema for text summarization."""
    text: str = Field(description="The text to summarize")

def create_structured_tools(groq_api_key: str) -> List[StructuredTool]:
    """Create structured tools for the agent."""

    search_arxiv_tool = StructuredTool.from_function(
        func=search_arxiv,
        name="search_arxiv",
        description="""Search for academic papers on arXiv. This tool returns a list of papers
            with their titles, IDs, authors, and URLs. Use this when you need to find papers on a topic.""",
        args_schema=SearchArxivInput
    )

    download_paper_tool = StructuredTool.from_function(
        func=download_paper,
        name="download_paper",
        description="""Download and extract the text content of an arXiv paper.
            Provide the paper ID (just the number part). Use this after finding
            papers with search_arxiv to read their content.""",
        args_schema=DownloadPaperInput
    )

    summarize_text_tool = StructuredTool.from_function(
        func=lambda text: summarize_text(text, groq_api_key),
        name="summarize_text",
        description="""Summarize a piece of text into key points.
            Use this to condense long paper content into digestible summaries.""",
        args_schema=SummarizeTextInput
    )

    return [search_arxiv_tool, download_paper_tool, summarize_text_tool]

# ---- Agent Setup ----

def create_research_agent(openai_api_key: str, groq_api_key: str) -> AgentExecutor:
    """Create a research agent using LangChain's AgentExecutor."""

    # Create the LLM
    llm = ChatOpenAI(api_key=openai_api_key, model="o3-mini")

    # Create the structured tools
    tools = create_structured_tools(groq_api_key)

    # Create the agent prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful AI research assistant. You have access to tools for searching academic papers,
            downloading paper content, and summarizing text.

            Your approach should be:
            1. Understand the research question
            2. Search for relevant papers using appropriate keywords
            3. Download and analyze the most relevant papers
            4. Synthesize information to answer the question

            Think step by step and make decisions about which tools to use when.
            When you've gathered enough information, provide a comprehensive answer."""),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    # Create the agent
    agent = create_openai_tools_agent(llm, tools, prompt)

    # Create agent executor
    return AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=10)

# ---- Custom Agent with Decision Logging ----

class ResearchAgentWithDecisions(AgentExecutor):
    """Extended AgentExecutor that logs decision points for educational purposes."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        object.__setattr__(self, 'decision_log', [])

    def _call(self, inputs: Dict[str, Any], *args, **kwargs) -> Dict[str, Any]:
        """Override _call to log decision points."""

        print("🤖 RESEARCH AGENT INITIATED")
        print(f"📋 Query: {inputs.get('input', '')}")

        # Call the parent class method
        result = super()._call(inputs, *args, **kwargs)

        print("\n✨ AGENT EXECUTION COMPLETE")
        print("\n----- DECISION LOG -----")
        for i, decision in enumerate(self.decision_log):
            print(f"{i+1}. {decision}")
        print("--------------------------")

        return result

    def _take_next_step(self, name_to_tool_map: Dict[str, Any], color_mapping: Dict[str, str],
                       inputs: Dict[str, Any], intermediate_steps: List[tuple], *args, **kwargs) -> Any:
        """Override to log agent decisions."""

        try:
            result = super()._take_next_step(name_to_tool_map, color_mapping, inputs, intermediate_steps, *args, **kwargs)

            # Log the decision
            if isinstance(result, list):
                for r in result:
                    if isinstance(r, tuple) and isinstance(r[0], AgentAction):
                        action = r[0]
                        decision = f"Decided to use tool: {action.tool} with input: {action.tool_input}"
                        self.decision_log.append(decision)
                        print(f"\n🔄 Agent Decision: {decision}")
            elif isinstance(result, AgentFinish):
                decision = f"Decided to finish with output: {result.return_values.get('output', '')[:100]}..."
                self.decision_log.append(decision)
                print(f"\n✅ Agent Finish: {decision}")

            return result

        except Exception as e:
            print(f"Error in agent execution: {e}")
            return AgentFinish(return_values={"output": "An error occurred during execution."}, log="Error")

# ---- Usage Example ----

def run_research_query(query: str, openai_api_key: str, groq_api_key: str) -> str:
    """Run a research query using the agent."""

    # Create the research agent
    agent_executor = ResearchAgentWithDecisions(
        agent=create_openai_tools_agent(
            ChatOpenAI(api_key=openai_api_key, model="o3-mini"),
            create_structured_tools(groq_api_key),
            ChatPromptTemplate.from_messages([
                ("system", """You are a helpful AI research assistant. You have access to tools for searching academic papers,
                    downloading paper content, and summarizing text.

                    Your approach should be:
                    1. Understand the research question
                    2. Search for relevant papers using appropriate keywords
                    3. Download and analyze the most relevant papers
                    4. Synthesize information to answer the question

                    Think step by step and make decisions about which tools to use when.
                    When you've gathered enough information, provide a comprehensive answer."""),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ])
        ),
        tools=create_structured_tools(groq_api_key),
        verbose=True,
        max_iterations=10
    )

    # Execute the query
    result = agent_executor.invoke({"input": query})

    return result["output"]

# ---- Example usage ----
def run_example(passed_query=""):
    # Load API keys from environment
    groq_api_key = os.getenv("GROQ_API_KEY")
    openai_api_key = os.getenv("OPENAI_API_KEY")

    if not groq_api_key or not openai_api_key:
        raise ValueError("API keys not found. Make sure GROQ_API_KEY and OPENAI_API_KEY are set.")

    # Example query
    # query = "What are the latest advancements in quantum error correction?"
    query = "What some progound insights in the best papers covering fine tuning in LLMs?"
    if passed_query:
        query = passed_query

    # Run the query
    response = run_research_query(query, openai_api_key, groq_api_key)

    print("\n----- FINAL ANSWER -----")
    print(response)
    print("\n--------------------------")

    return response

In [32]:
run_example(passed_query="What some profound insights in the best papers covering fine tuning in LLMs?")



> Entering new ResearchAgentWithDecisions chain...
🤖 RESEARCH AGENT INITIATED
📋 Query: What some progound insights in the best papers covering fine tuning in LLMs?
[{'id': '2403.09162v1', 'title': 'Unveiling the Generalization Power of Fine-Tuned Large Language Models', 'authors': ['Haoran Yang', 'Yumeng Zhang', 'Jiaqi Xu', 'Hongyuan Lu', 'Pheng Ann Heng', 'Wai Lam'], 'url': 'http://arxiv.org/pdf/2403.09162v1'}, {'id': '2309.13192v2', 'title': 'Towards Green AI in Fine-tuning Large Language Models via Adaptive Backpropagation', 'authors': ['Kai Huang', 'Hanyun Yin', 'Heng Huang', 'Wei Gao'], 'url': 'http://arxiv.org/pdf/2309.13192v2'}, {'id': '2310.10047v1', 'title': 'Improving Large Language Model Fine-tuning for Solving Math Problems', 'authors': ['Yixin Liu', 'Avi Singh', 'C. Daniel Freeman', 'John D. Co-Reyes', 'Peter J. Liu'], 'url': 'http://arxiv.org/pdf/2310.10047v1'}, {'id': '2408.13296v3', 'title': 'The Ultimate Guide to Fine-Tuning LLMs from Basics to Breakthroughs: An Exha

"I reviewed some of the influential recent works on fine-tuning in large language models (LLMs), and several profound insights have emerged across these papers. Here are some of the key takeaways:\n\n1. Deep Understanding of the Fine-Tuning Pipeline:\n   • Many of the best papers emphasize that fine-tuning is not just “tweaking” a model’s weights but involves a comprehensive, multi-stage pipeline—from careful data preparation and handling imbalanced datasets to model initialization, hyperparameter tuning, and validation. For example, in “The Ultimate Guide to Fine-Tuning LLMs from Basics to Breakthroughs,” a seven-stage pipeline is introduced that underscores the importance of each step in ensuring robust performance.\n\n2. Parameter-Efficient Fine-Tuning:\n   • With the growing size of LLMs, the cost and computational burden of fine-tuning a full model becomes prohibitively high. As a result, techniques like Low-Rank Adaptation (LoRA) and Half Fine-Tuning have gained traction. These a